# CRDS reference rules

This notebook explores the CRDS .rmap rules files.

## Prerequisites

To follow along with the examples in this notebook, you will need:

- Packages from the requirements.txt included in this notebook's directory:

```
$ pip install -r requirements.txt
```

## Setup

We can connect to the JWST ops server in this notebook, since we won't be attempting to submit changes to the system.

In [ ]:
import os
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
os.environ["CRDS_PATH"] = os.path.join(os.environ["HOME"], "crds-tutorial-cache")
import crds # Always delay import until environment variables are set.

## The .rmap language

We've encountered .rmap rules files in previous notebooks, but now we'll delve into them in more detail.  An .rmap is written in a little language all its own -- a subset of Python where certain classes are assumed to be imported before the .rmap "script" is interpreted.  An .rmap defines two required and one optional top-level variable.  Let's discuss each variable one at a time, starting with `header`.

### header

The required `header` variable contains metadata for the .rmap itself.  We can use the CRDS client to pull up a real example of an .rmap and examine its header.  Let's grab the latest (at time of writing) `nircam` `dark` .rmap:

In [ ]:
# Make sure mappings have been cached locally:
crds.client.api.dump_mappings("jwst_0641.pmap")

# Get the path to the .rmap in the local cache:
path = crds.locate_mapping("jwst_nircam_dark_0021.rmap")

# Read the .rmap content and print:
with open(path) as f:
    nircam_dark_rmap = f.read()
    
print(nircam_dark_rmap)

Certain header keys are required in every .rmap, while others enable optional features.  The required keys:

#### derived_from

The `derived_from` key documents the "heritage" of the .rmap file.  For brand-new .rmaps, the value will be a descriptive string like "Created by hand on 2020-09-15".  For an .rmap that is an update to a previously existing set of rules, the `derived_from` value will be the original filename.  In our example, the key is `jwst_nircam_dark_0018.rmap`, showing that our .rmap is an update to a previous version.

The `derived_from` value is automatically maintained by CRDS when an .rmap update is generated by the system in response to a reference file submission.  When an .rmap is updated by hand, `derived_from` must be updated manually.

`derived_from` is not used when evaluating the rules, but it is important for understanding how a set of rules came to be.

#### filekind

This is simply the reference file type, in this case, `DARK`.  The type string is not case sensitive, but by convention .rmaps are written with `filekind` in uppercase.

#### instrument

The instrument whose reference files the .rmap describes.  We're looking at a `nircam` .rmap, so the value is `NIRCAM`.

#### mapping

This key identifies the fact that these are reference rules, as opposed to instrument (.imap) or top-level context (.pmap) rules.  We're looking at an .rmap, so this value is `REFERENCE`.

#### name

The name of this file, which also serves as its unique identifier in CRDS.  Given such a filename, CRDS knows how to build the full path to the file in the CRDS cache directory structure.

The `name` value is automatically maintained by CRDS when an .rmap update is generated by the system, and when CRDS is asked to assign a name to a manual update.

#### observatory

The observatory whose reference files the .rmap describes.  We're looking at a JWST .rmap, so the value is `JWST`.

#### parkey

A tuple of tuples that describes selector criteria for each tier of the selector tree.  `parkey` informs CRDS as to which dataset metadata fields are needed to evaluate the .rmap's rules.  Typically the outer tuple will be of length 1 (for non-time-dependent matching) or length 2 (for time-dependent matching), but other configurations are possible.  In our example .rmap, `parkey` describes two selector tiers:

```
'parkey' : (('META.EXPOSURE.TYPE', 'META.INSTRUMENT.DETECTOR', 'META.SUBARRAY.NAME'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
```

The first nested tuple shows that exposure type, detector, and subarray name are needed to evaluate the first selector tier.  The second nested tuple shows that observation date and time are needed to evaluate the second selector tier.

The meaning of parkey will become more clear in the section on selectors below.

#### sha1sum

The SHA-1 checksum of this .rmap file.  This is a checksum of every line of the .rmap except the `sha1sum` line (you'd be hard-pressed to compute a checksum that includes itself!).  This key is automatically maintained by CRDS for both auto-generated and hand-crafted .rmaps, but you'll see a warning on submission of the checksum doesn't match your manual .rmap.

In the exercises at the end of this notebook, we pass a flag to CRDS instructing it to ignore checksum mismatches, so you won't need to update this value.

#### Optional keys

Optional header features are outside the scope of this notebook, but you can learn about them by reading the [CRDS User Manual](https://jwst-crds-bit.stsci.edu/static/users_guide/rmap_syntax.html).  The `substitutions` key in our example .rmap allows CRDS to treat the string `GENERIC` like the special wildcard value `N/A` when found in the dataset's `META.SUBARRAY.NAME` field.

#### Note on custom keys

The header also permits arbitrary additional keys, so you may find .rmaps (particularly older specimens) that include keys that you don't recognize and aren't documented in the CRDS User Manual.  Some of them are custom metadata that the original .rmap author thought might be useful, others refer to header features that have since been removed.  They will not impact the evaluation of the .rmap's rules.

### comment

The optional `comment` variable is just that: a freeform string that can be used for documentation purposes.

### selector

The required `selector` variable contains the actual reference file selection rules that are the .rmap's reason for being.  Here is encoded the logic of the file selection process for every file of a particular instrument's reference file type.  No two instruments are handled by the same .rmap, and no instrument + reference type combination spans multiple .rmaps.

The selector is arranged in a tree structure, which takes the form of composed Python objects.  Objects at a given tier of the tree receive input as described by the corresponding tuple in the header `parkey`.  The 1st tier of the tree pairs with the 1st inner tuple, and so on.

Selector objects are initialized with a dict, where the expected key type depends on selector type.  The values are string filenames, the special value `'N/A'`, or additional selectors.  Many selector types are available, but the most commonly used are `{}`, `Match`, and `UseAfter`.  None of these need be imported in the .rmap "script"; they can be assumed to always be available.

#### {} selector

The `{}` selector performs simple exact matches on a single dataset metadata value.  For example, with `parkey` value `(('META.INSTRUMENT.DETECTOR',),)`, the following selector chooses a different file for each of two detectors:

```
selector = {
  'NCRA1': 'the_file_for_ncra1.asdf',
  'NCRA2': 'the_file_for_ncra2.asdf',
}
```

The `{}` selector is commonly seen in .pmap and .imap files, but less so in .rmap files, where the more advanced features of the `Match` selector tend to be necessary.

#### Match selector

The `Match` selector has many advantages over the simpler `{}`: it can match on multiple dataset metadata fields at once, offers wildcard, regular expression, and logical-or matches, chooses from among multiple matching rules using a system of weights, and more.

Instances of the `Match` selector are initialized with a dict, where the keys are tuples containing 1..N dataset metadata values (or patterns, or wildcards).  For example, with `parkey` value `(('META.INSTRUMENT.DETECTOR', 'META.EXPOSURE.TYPE'),)`, the following selector chooses different files for various combinations of detector and exposure type:

```
selector = Match({
  ('NCRA1', 'NRC_CORON'): 'file_0001.asdf',
  ('NCRA1', 'NRC_FLAT'): 'file_0002.asdf',
  ('NCRA2', 'NRC_CORON'): 'file_0003.asdf',
  ('NCRA2', 'NRC_FLAT'): 'file_0004.asdf',
})
```

The majority of .rmaps will use the `Match` selector, some in a single tier configurations, but most in two tiers with a nested `UseAfter` selector within each `Match` rule.

#### UseAfter selector

The `UseAfter` selector implements matching on the dataset's observation timestamp, when reference files are only appropriate to "use" with observations taken "after" a certain date and time.  Instances of the `UseAfter` selector are initialized with a dict, where the keys are timestamps in the format `YYYY-MM-DD hh:mm:ss`.  The selector matches the the latest timestamp that is still on or before the observation timestamp.

For example, with `parkey` value `(('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME'),)` the following selector chooses different files depending on the observation timestamp:

```
selector = UseAfter({
  '2020-01-01 00:00:00': '2020_file.asdf',
  '2021-01-01 00:00:00': '2021_file.asdf',
  '2022-01-01 00:00:00': '2022_file.asdf',
})
```

A dataset with observation timestamp `2021-01-01 00:00:00` would select `2021_file.asdf`, as would a dataset with timestamp `2021-12-31 23:59:59`.  A dataset with timestamp `2019-12-31 23:59:59` wouldn't select anything, and any timestamp on or after `2022-01-01 00:00:00` (including timestamps in the year 9000) would select `2022_file.asdf`.

#### Dissecting the example selector

Now to return to our `nircam` `dark` .rmap.  Let's print it out again, since at this point it has floated away up the page:

In [ ]:
print(nircam_dark_rmap)

Here we have a two-tiered selector.  The first tier is a `Match` selector on three dataset metadata fields, as we can see in the header `parkey`: `META.EXPOSURE.TYPE`, `META.INSTRUMENT.DETECTOR`, and `META.SUBARRAY.NAME`.  The second tier is a `UseAfter` selector on the observation date and time fields.  Because `UseAfter` is the second tier, the `Match` values are mostly `UseAfter` instances.

The first entry right away uses four `Match` features that aren't available in a simple `{}` selector.  For one thing, it's matching three metadata fields at once.  Then there's the special value `N/A`, which is a wildcard that matches any exposure type, but with less weight than other matches (so it will tend to be selected only if a more specific match isn't available).  The special value `ANY` is also a wildcard, so it matches any detector, but it is assigned full weight.  The `SUB8FP1A|SUB8FP1B` uses the logical-or operator `|` so either of the two subarrays will match.  The value of this match rule, `'N/A'`, indicates that no reference file should be returned in this case.

In English, this rule might be described like this: for any exposure type and detector that was used in conjunction with subarrays SUB8FP1A or SUB8FP1B, return no reference file.  However, another match rule may override this rule for specific exposure types.

The second entry also starts with a weak wildcard match on exposure type, but continues with specific matches on the `NRCA1` detector and the `FULL` subarray.  It then proceeds to the `UseAfter` selector, and selects the `jwst_nircam_dark_0040.fits` file if the observation timestamp is 20th century or later.  It's common practice to choose an absurdly early `UseAfter` date to effectively cover all time.

In English, the second rule might be described like this: for any exposure type with the `NCRA1` detector and the `FULL` subarray, select the `jwst_nircam_dark_0040.fits` reference file for observations taken at (effectively) any time.

In the next section, we'll try our hand at writing some .rmap rules.

## Reference rules olympics

Congratulations, you've been chosen to represent your scrum team in the .rmap olympics!  For each of the following challenges, edit the `RMAP` variable according to the instructions.  Test your changes by executing the "Test your solution" code cell.

### Challenge 1

To celebrate your humble notebook author's birthday, we're going to change the NIRCam `photom` reference file to a special edition just for that day.  Modify the following .rmap to select `jwst_nircam_photom_birthday.fits` for the duration of 11/20/2020, but only for the `NRC_IMAGE` exposure type.

In [ ]:
RMAP = """
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_nircam_photom_0009.rmap',
    'filekind' : 'PHOTOM',
    'instrument' : 'NIRCAM',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_nircam_photom_0010.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'sha1sum' : '32715a282c1417c74b06eaac369d1c1bb99ba248',
}

selector = Match({
    ('NRCA1', 'N/A') : UseAfter({
        '1900-01-01 00:00:00' : 'jwst_nircam_photom_0031.fits',
        '2014-01-01 00:00:00' : 'jwst_nircam_photom_0048.fits',
    }),
    ('NRCA1', 'NRC_CORON|NRC_FLAT|NRC_FOCUS|NRC_IMAGE|NRC_TACONFIRM|NRC_TACQ|NRC_TSIMAGE') : UseAfter({
        '2014-01-01 00:00:00' : 'jwst_nircam_photom_0074.fits',
    }),
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(exposure_type, date, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": "NRCA1",
            "META.EXPOSURE.TYPE": exposure_type,
            "META.OBSERVATION.DATE": date,
            "META.OBSERVATION.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.EXPOSURE.TYPE={exposure_type}, META.OBSERVATION.DATE={date}.  Expected {filename}, got {result}"
        raise AssertionError(message)
    
assert_best_ref("SOME EXPOSURE TYPE", "1980-01-01", "jwst_nircam_photom_0031.fits")
assert_best_ref("SOME EXPOSURE TYPE", "2014-01-01", "jwst_nircam_photom_0048.fits")
assert_best_ref("SOME EXPOSURE TYPE", "2020-11-20", "jwst_nircam_photom_0048.fits")


for exposure_type in ("NRC_CORON", "NRC_FLAT", "NRC_FOCUS", "NRC_IMAGE", "NRC_TACONFIRM", "NRC_TACQ", "NRC_TSIMAGE"):
    assert_best_ref(exposure_type, "2014-01-01", "jwst_nircam_photom_0074.fits")
    assert_best_ref(exposure_type, "2020-11-21", "jwst_nircam_photom_0074.fits")
    
for exposure_type in ("NRC_CORON", "NRC_FLAT", "NRC_FOCUS", "NRC_TACONFIRM", "NRC_TACQ", "NRC_TSIMAGE"):
    assert_best_ref(exposure_type, "2020-11-20", "jwst_nircam_photom_0074.fits")

assert_best_ref("NRC_IMAGE", "2020-11-20", "jwst_nircam_photom_birthday.fits")

print("Success!")

### Challenge 2

Oh no, MIRI was struck by rogue fireworks and the previously stable flat field correction completely changed!  Add time-dependence to the following .rmap.  For each match case, use the existing reference file up to 10pm UTC on 7/4/2024, but switch to the following files (identified by band) at that time:

`LONG`: jwst_miri_flat_0600.fits

`MEDIUM`: jwst_miri_flat_0601.fits

`SHORT`: jwst_miri_flat_0602.fits

In [ ]:
RMAP = """
header = {
    'classes' : ('Match',),
    'derived_from' : 'jwst_miri_flat_0045.rmap',
    'filekind' : 'FLAT',
    'instrument' : 'MIRI',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_miri_flat_0046.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.INSTRUMENT.FILTER', 'META.INSTRUMENT.BAND', 'META.EXPOSURE.READPATT', 'META.SUBARRAY.NAME'),),
    'sha1sum' : '1b42da81d62fb32d927911f3dcae05a980bcf939',
}

selector = Match({
    ('MIRIFULONG', 'N/A', 'LONG', 'N/A', 'FULL') : 'jwst_miri_flat_0541.fits',
    ('MIRIFULONG', 'N/A', 'MEDIUM', 'N/A', 'FULL') : 'jwst_miri_flat_0539.fits',
    ('MIRIFULONG', 'N/A', 'SHORT', 'N/A', 'FULL') : 'jwst_miri_flat_0542.fits',
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(band, date, time, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": "MIRIFULONG",
            "META.INSTRUMENT.BAND": band,
            "META.SUBARRAY.NAME": "FULL",
            "META.OBSERVATION.DATE": date,
            "META.OBSERVATION.TIME": time,
        }
    )
    
    if result != filename:
        message = f"Test failed for META.EXPOSURE.BAND={band}, META.OBSERVATION.DATE={date}, META.OBSERVATION.TIME={time}.  Expected {filename}, got {result}"
        raise AssertionError(message)
        
assert_best_ref("LONG", "2023-01-01", "00:00:00", "jwst_miri_flat_0541.fits")
assert_best_ref("LONG", "2024-07-04", "21:59:59", "jwst_miri_flat_0541.fits")
assert_best_ref("LONG", "2024-07-04", "22:00:00", "jwst_miri_flat_0600.fits")
assert_best_ref("LONG", "2025-01-01", "00:00:00", "jwst_miri_flat_0600.fits")

assert_best_ref("MEDIUM", "2023-01-01", "00:00:00", "jwst_miri_flat_0539.fits")
assert_best_ref("MEDIUM", "2024-07-04", "21:59:59", "jwst_miri_flat_0539.fits")
assert_best_ref("MEDIUM", "2024-07-04", "22:00:00", "jwst_miri_flat_0601.fits")
assert_best_ref("MEDIUM", "2025-01-01", "00:00:00", "jwst_miri_flat_0601.fits")

assert_best_ref("SHORT", "2023-01-01", "00:00:00", "jwst_miri_flat_0542.fits")
assert_best_ref("SHORT", "2024-07-04", "21:59:59", "jwst_miri_flat_0542.fits")
assert_best_ref("SHORT", "2024-07-04", "22:00:00", "jwst_miri_flat_0602.fits")
assert_best_ref("SHORT", "2025-01-01", "00:00:00", "jwst_miri_flat_0602.fits")

print("Success!")

### Challenge 3

Construct a selector for the `nirspec` `disperser` reference type with the following rules:

- If the exposure type is `NRS_DARK`, return reference file not applicable.  This rule overrides any subsequent rule.
- If the grating is `G140H` and the exposure type one of (`NRS_AUTOWAVE`, `NRS_AUTOFLAT`, or `NRS_TACQ`), return `jwst_nirspec_disperser_0001.asdf`.
- If the grating is `G140H` and the exposure type not already covered by a previous rule, return `jwst_nirspec_disperser_0002.asdf`.
- If the grating is `MIRROR`, return `jwst_nirspec_disperser_0003.asdf` regardless of exposure type (except `NRS_DARK`).

The selector should be time-dependent, but this initial set of files can all share a useafter timestamp of `2020-01-01 00:00:00`.

In [ ]:
RMAP = """
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_nirspec_disperser_0019.rmap',
    'filekind' : 'DISPERSER',
    'instrument' : 'NIRSPEC',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_nirspec_disperser_0019.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.GRATING', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'sha1sum' : 'ad74383edba73deabd88922565b5f8da7237d779',
}

selector = Match({
    # ???
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(grating, exposure_type, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.GRATING": grating,
            "META.EXPOSURE.TYPE": exposure_type,
            "META.OBSERVATION.DATE": "2020-01-01",
            "META.OBSERVATION.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.INSTRUMENT.GRATING={grating}, META.EXPOSURE.TYPE={exposure_type}.  Expected {filename}, got {result}"
        raise AssertionError(message)
        
for grating in ["G140H", "MIRROR", "PRISM"]:
    assert_best_ref(grating, "NRS_DARK", "NOT FOUND n/a")
    
for exposure_type in ["NRS_AUTOWAVE", "NRS_AUTOFLAT", "NRS_TACQ"]:
    assert_best_ref("G140H", exposure_type, "jwst_nirspec_disperser_0001.asdf")
    
import random
import string
random_exposure_type = "NRS_" + "".join([random.choice(string.ascii_uppercase) for _ in range(8)])

assert_best_ref("G140H", random_exposure_type, "jwst_nirspec_disperser_0002.asdf") 

for exposure_type in ["NRS_AUTOWAVE", "NRS_AUTOFLAT", "NRS_TACQ", random_exposure_type]:
    assert_best_ref("MIRROR", exposure_type, "jwst_nirspec_disperser_0003.asdf")
    assert_best_ref("PRISM", exposure_type, "NOT FOUND No match found.")
    
print("Success!")

### Challenge 4

Construct a selector for the `nirspec` `dflat` reference type with the following rules:

- For the `NRS1` detector and any exposure type that begins with `NRS_` select the `jwst_nirspec_dflat_0001.fits` reference file.  Other exposure types should select `jwst_nirspec_dflat_0002.fits`.
- For the `NRS2` detector and the following exposure types: `NRS_MSATA`, `NRS_WATA`, `NRS_CONFIRM`, select `jwst_nirspec_dflat_0003.fits`.  For `NRS_FOCUS`, select `jwst_nirspec_dflat_0004.fits`.  Other exposure types should select `jwst_nirspec_dflat_0005.fits`.

The selector should be time-dependent, but this initial set of files can all share a useafter timestamp of `2020-01-01 00:00:00`.

Hint: you will need to use a `Match` feature that hasn't been discussed in this notebook.  Consult the [CRDS User Manual](https://jwst-crds-bit.stsci.edu/static/users_guide/rmap_syntax.html) for assistance.

In [ ]:
RMAP = """
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_nirspec_dflat_0005.rmap',
    'filekind' : 'DFLAT',
    'instrument' : 'NIRSPEC',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_nirspec_dflat_0005.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'sha1sum' : '3ee07dcd1fe41ad6d5e4e1b4dbcf3eaa4369659e',
}

selector = Match({
    # ???
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)

def assert_best_ref(detector, exposure_type, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": detector,
            "META.EXPOSURE.TYPE": exposure_type,
            "META.OBSERVATION.DATE": "2020-01-01",
            "META.OBSERVATION.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.INSTRUMENT.DETECTOR={detector}, META.EXPOSURE.TYPE={exposure_type}.  Expected {filename}, got {result}"
        raise AssertionError(message)

import random
import string
random_nrs_exposure_type = "NRS_" + "".join([random.choice(string.ascii_uppercase) for _ in range(random.randint(4, 30))])
random_other_exposure_type = "".join([random.choice(string.ascii_uppercase) for _ in range(random.randint(4, 30))])
        
for exposure_type in ["NRS_MSATA", "NRS_WATA", "NRS_CONFIRM", random_nrs_exposure_type]:
    assert_best_ref("NRS1", exposure_type, "jwst_nirspec_dflat_0001.fits")
    
assert_best_ref("NRS1", random_other_exposure_type, "jwst_nirspec_dflat_0002.fits")
    
for exposure_type in ["NRS_MSATA", "NRS_WATA", "NRS_CONFIRM"]:
    assert_best_ref("NRS2", exposure_type, "jwst_nirspec_dflat_0003.fits")

assert_best_ref("NRS2", "NRS_FOCUS", "jwst_nirspec_dflat_0004.fits")

for exposure_type in [random_nrs_exposure_type, random_other_exposure_type]:
    assert_best_ref("NRS2", exposure_type, "jwst_nirspec_dflat_0005.fits") 
    
print("Success!")

### Challenge 5

You're throwing a series of dinner parties!  You've stored your favorite recipes in individual ASDF files and need an .rmap that will select them based on metadata from your guest lists.  A guest list has the following metadata available:

**META.PARTY.GUEST_COUNT**: A positive integer.

**META.PARTY.DIET**: One of `VEGAN`, `VEGETARIAN`, or `OMNIVORE`.

**META.PARTY.ALLERGEN**: A string identifier for a single allergen.  The full set of allergens is unknown, but you're confident that your recipe metadata is complete.

**META.PARTY.DATE**: The date of the party.

**META.PARTY.TIME**: The time of the party.

Here are your recipes:

**mac_and_cheese.asdf**
```
Servings: 100
Allergens: WHEAT, DAIRY
Incompatible diets: VEGAN
Seasonal availability: (unrestricted)
```

**mac_and_cheese_with_asparagus.asdf**
```
Servings: 100
Allergens: WHEAT, DAIRY
Incompatible diets: VEGAN
Seasonal availability: February 1st, 2022 until June 1st, 2022
```

**stir_fry_with_beef.asdf**
```
Servings: 4
Allergens: (none)
Incompatible diets: VEGAN, VEGETARIAN
Seasonal availability: (unrestricted)
```

**oatmeal.asdf**
```
Servings: (unlimited)
Allergens: (none)
Incompatible diets: (none)
Seasonal availability: (unrestricted)
```

**oatmeal_with_apples.asdf**
```
Servings: (unlimited)
Allergens: (none)
Incompatible diets: (none)
Seasonal availability: August 1st, 2022 until November 1st, 2022
```

Serving oatmeal should be avoided whenever possible.  Prioritize seasonal vegetables.  In anticipation of continued SARS-CoV-2 restrictions, the parties are planned for the year 2022.

In [ ]:
RMAP = """
header = {
    # ???
}

selector = Match({
    # ???
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP, ignore_checksum=True)
def assert_best_ref(guest_count, diet, allergen, date, filename):
    result = rmap.get_best_ref(
        {
            "META.PARTY.GUEST_COUNT": str(guest_count),
            "META.PARTY.DIET": diet,
            "META.PARTY.ALLERGEN": allergen,
            "META.PARTY.DATE": date,
            "META.PARTY.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.PARTY.GUEST_COUNT={guest_count}, META.PARTY.DIET={diet}, META.PARTY.ALLERGEN={allergen}, META.PARTY.DATE={date}.  Expected {filename}, got {result}"
        raise AssertionError(message)
               
for guest_count in [101, 1000, 10000]:
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-07-01", "oatmeal.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-08-15", "oatmeal_with_apples.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-11-20", "oatmeal.asdf")
    
for guest_count in range(5, 100):
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-01-05", "mac_and_cheese.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "N/A", "2022-03-08", "mac_and_cheese_with_asparagus.asdf")
    assert_best_ref(guest_count, "VEGAN", "N/A", "2022-09-20", "oatmeal_with_apples.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "WHEAT", "2022-09-20", "oatmeal_with_apples.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "DAIRY", "2022-09-20", "oatmeal_with_apples.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "KANGAROO", "2022-01-05", "mac_and_cheese.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "KANGAROO", "2022-03-08", "mac_and_cheese_with_asparagus.asdf")
    
for guest_count in range(1, 5):
    assert_best_ref(guest_count, "OMNIVORE", "WHEAT", "2022-05-01", "stir_fry_with_beef.asdf")
    assert_best_ref(guest_count, "OMNIVORE", "DAIRY", "2022-05-01", "stir_fry_with_beef.asdf")
    assert_best_ref(guest_count, "VEGETARIAN", "WHEAT", "2022-05-01", "oatmeal.asdf")
    assert_best_ref(guest_count, "VEGAN", "WHEAT", "2022-05-01", "oatmeal.asdf")
    
print("Success!")

## Further reading

The CRDS User Manual includes [detailed documentation](https://jwst-crds-bit.stsci.edu/static/users_guide/rmap_syntax.html) on the subject of CRDS rules files.